# [문제]
- law_2.docx 파일을 읽고, Chroma 저장
- LLM 질문 -> 답변
- 전세사기피해에 관한 법률 질문만 받기
- 이 외의 질문은 '답변을 할 수 없습니다.' 

In [1]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import Docx2txtLoader


## 1. 문서 내용 읽고 분할
loader = Docx2txtLoader('law_2.docx')

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

document_list = loader.load_and_split(text_splitter=text_splitter)


## 2. 임베딩 -> 벡터 데이터베이스에 저장
## 2.1. 환경변수 읽어오기
load_dotenv()

## 2.2. 임베딩 모델 지정
embedding = OpenAIEmbeddings(model='text-embedding-3-large')

## 2.3. 벡터 데이터베이스에 저장
## [방법 1] in memory
database = Chroma.from_documents(
    documents=document_list,
    embedding=embedding,
)


## 3. 질문 -> 벡터 데이터베이스(vector store)에서 유사도 검색
## 3.1. 사용자 질문
query = '전세사기피해자는 누구인가요?'
# query = '오늘 점심 메뉴 정해줘~'

## 3.2. 벡터 데이터베이스에서 유사도 검색
retrieved_docs = database.similarity_search(query=query, k=2)

## 3.3. 문서 객체 2개 -> 하나의 문자열
context = '😊😊\n\n'.join([doc.page_content for doc in retrieved_docs])


## 4. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달
## 4.1. 프롬프트 작성
prompt = '''
[identity]
- 당신은 전세사기피해 법률 전문가입니다.
- [context]를 참고하여 사용자의 질문에 답변해주세요.
- 전세사기피해 법률 이외의 질문에는 '답변을 할 수 없습니다.'로 답하세요.

[context]
{retrieved_docs}

Question: {query}
'''

## 4.2. 프롬프트 변수에 값 설정
formmated_prompt = prompt.format(
    retrieved_docs=context,
    query=query,
)

## 4.3. LLM 모델 생성(ChatOpenAI 인스턴스 생성)
llm = ChatOpenAI(model='gpt-4o')

## 4.4. LLM 모델에 질문과 검색된 문서 보내기
ai_message = llm.invoke(formmated_prompt)
ai_message.content

'「전세사기피해자 지원 및 주거안정에 관한 특별법」에 따르면 전세사기피해자는 임대인에 의해 임대차 계약의 내용이 사기 또는 기만으로 인해 손해를 입은 임차인을 말합니다. 주로 계약 체결 당시 임대인의 의도적 기만행위나 계약 이행의 불이행 등으로 인한 피해를 입은 경우가 해당됩니다. 이 법에서는 이러한 피해자들을 지원하고 그들의 주거 안정을 도모하기 위한 다양한 지원 방안을 규정하고 있습니다.'